In [1]:
import torch
# neural network 
import torch.nn as nn
# 최적화 - 함수에 대한 최대 혹은 최저값의 변수들은 찾는 것 
import torch.optim as optim
#ann이라는 모델을 다를 때 f(ax+b) 이미지 처리
import torchvision
#전처리를 위한 lib
import torchvision.transforms as transforms

In [3]:
#데이터셋 전처리
# 학습을 원활히 하기 위한 과정으로 transform 여러 변환을 순차적으로 적용, 
# 평균이 0.5 표준편차 0.5로 이미지 정규화
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# MNIST 데이터셋로드
# batch_size64개로 쪼개 순서대로 쪼개면 순서관계가 있는경우 의존성이 있을수 있어 학습이 잘되지 않을수 있어 섞는다.
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████| 9912422/9912422 [00:05<00:00, 1716543.97it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████| 28881/28881 [00:00<00:00, 140027.83it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████| 1648877/1648877 [00:01<00:00, 1271244.68it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████| 4542/4542 [00:00<00:00, 892630.90it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [5]:
trainloader

딥러닝 모델을 만드는 과정
클래스 정의 nn모듈 상속받아야 한다. 
딥러닝 모델을 만들기 위한 기본기능을 사전에 가져온다.
1. 생성자 init함수 구성 - 각각의 layer정의
2. layer 구성 fc 는 fully connected layer - layer간의 모두 연결되어 있을때 fc
3. 파라미터로 모델 설정 nn.linear - ann모델을 만들기 위한 함수 layer에서 입력과 출력 정의해야 한다.
4. fc1의 두번째 파라미터 fc2의 첫번째 파라미터와 연결, fc2의 두번째 파라미터 fc3의 첫번째 파라미터와 연결 0-9까지 예측하기 위해 마지막 fc3의 두번째 파라미터는 10개의 퍼셉트론
5. 28*28은 mnist의 데이터 크기 사진을 받아서 처리하고 최종 10개로 예측
6. forward함수정의 - 자동적으로 layer간의 연결관계 정의. 이미지 데이터가 들어오는데 fully connected layer는 1차원데이터를 받는 layer이므로 우선 1차원으로 변환 view함수를 통해 차원변환. -1은 2차원을 기준으로 데이터를 알아서 정의하라는 의미.
7. 데이터가들어올때 batch단위인 64개 쪼개서 들어오는데 학습을 위한 데이터 1*64 데이터차원을지정해주고 batch의 차원을 알아서 정의한다.
8. torch.relu(self.fc1) - 입력레이어에 전달한다음 relu저장 x 64*10 

In [7]:
class SimpleANN(nn.Module):
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128) #입력층에서 은닉층으로
        self.fc2 = nn.Linear(128, 64)    #은닉층에서 은닉층으로
        self.fc3 = nn.Linear(64, 10)     #은닉층에서 출력층으로

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [9]:
# 모델 초기화
model = SimpleANN()

# 손실함수 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 모델 학습
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # 기울기 초기화, no_grad 평가단계에서 오차함수를 줄이는 방향으로 업데이트 하기 위해서 미분을 하는데 즉,기울기 계산하지않는다
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 출력
        running_loss = running_loss + loss.item()
        if i%100 == 99:
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0
print('Finished Training')

[Epoch 1, Batch 100] loss: 1.326
[Epoch 1, Batch 200] loss: 0.450
[Epoch 1, Batch 300] loss: 0.385
[Epoch 1, Batch 400] loss: 0.353
[Epoch 1, Batch 500] loss: 0.310
[Epoch 1, Batch 600] loss: 0.300
[Epoch 1, Batch 700] loss: 0.274
[Epoch 1, Batch 800] loss: 0.251
[Epoch 1, Batch 900] loss: 0.240
[Epoch 2, Batch 100] loss: 0.199
[Epoch 2, Batch 200] loss: 0.194
[Epoch 2, Batch 300] loss: 0.180
[Epoch 2, Batch 400] loss: 0.197
[Epoch 2, Batch 500] loss: 0.160
[Epoch 2, Batch 600] loss: 0.172
[Epoch 2, Batch 700] loss: 0.164
[Epoch 2, Batch 800] loss: 0.154
[Epoch 2, Batch 900] loss: 0.175
[Epoch 3, Batch 100] loss: 0.129
[Epoch 3, Batch 200] loss: 0.138
[Epoch 3, Batch 300] loss: 0.137
[Epoch 3, Batch 400] loss: 0.130
[Epoch 3, Batch 500] loss: 0.115
[Epoch 3, Batch 600] loss: 0.129
[Epoch 3, Batch 700] loss: 0.123
[Epoch 3, Batch 800] loss: 0.125
[Epoch 3, Batch 900] loss: 0.114
[Epoch 4, Batch 100] loss: 0.107
[Epoch 4, Batch 200] loss: 0.103
[Epoch 4, Batch 300] loss: 0.094
[Epoch 4, 

In [12]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total = total+labels.size(0)
        correct = correct + (predicted == labels).sum().item()
        # print(images, labels)

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the network on the 10000 test images: 97.35%


배운내용
학습을 위해 손실 함수, 모델 정의, 역전파 사용, 가중치 업데이트, 데이터 쪼개는 방법 

손실함수 - 예측값, 실제값의 차이, DL이 학습이 잘되는 형태로 구성, 단순히 오차만 계산하는것이 아니고 오차가 어떻게 학습이 잘되는 형태인지 나타낸다. 모델의 성능을 평가 가능.